TO DO:
- Analyze sub clusters and look at their performance 
- Change metric when calculating clusters (currently ward) 
- Change linkage metric when calculating 
- Check with professor that euclidean makes sense because tf-idf is normalized    
    
TO DO in general:
- score and rank clusters 
- metrics to compare clusters for two methods
- Visualization with tsne or umap rather than mds -- tsne works. can't get umap to install.   

In [1]:
import nltk
from nltk.corpus import reuters
import re
import numpy as np
import pandas as pd
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
from sklearn.manifold import MDS
import sklearn
from scipy.cluster.hierarchy import dendrogram, linkage, ward, fcluster
import networkx as nx
import collections
import math
import operator
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.decomposition import PCA
from kneed import KneeLocator
from sklearn.manifold import TSNE
import scipy.spatial.distance

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gimli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Hierarchial Clustering

### Tf-Idf Matrix

In [2]:
def tf_idf(df):
    # tfidf. stop word removal. word tokenizer. 
    tfidf = TfidfVectorizer(stop_words = 'english', analyzer = 'word')
    m = tfidf.fit_transform(df['text'])
    
    feature_names = tfidf.get_feature_names() # words 

    return m, feature_names

### Distance Matrix

In [3]:
def dist_calculate(m):
    dist = euclidean_distances(m)  ## I think its ok to use euclidean because tf-idf normalizes
    flat_dist = scipy.spatial.distance.pdist(m, 'euclidean') # needed for linkage function
    # euclidean can be innaccurate if documents are different lengths such that vectors are different lengths 
    # I would prefer to use euclidean because then more sensicl to calculate centroids
    # ask professor?? 
    return dist, flat_dist

### PCA Dimensionality Reduction

In [4]:
def pca_reduce(m):
    pca = PCA(n_components = 0.8) # keep 95% of variance 
    pcam = pca.fit_transform(m.toarray())

    return pcam

### Linkage Matrix

In [5]:
def linkage_calculate(dist):
    linkage_matrix = linkage(dist, method = 'ward') 
    return linkage_matrix
    
# plot dendogram
#fig, ax = plt.subplots(figsize=(15, 20))
#ax = dendrogram(linkage_matrix, orientation="right", labels = df_retail.ids.unique())

### Choose K and Create Clusters

In [6]:
def frame_merge(df, f):
    # merge in with original data via pandas
    frameh = pd.DataFrame(df.index, index = [f], columns = ['index_search'])
    frameh = pd.merge(frameh, df, right_index = True, left_on = 'index_search')
    frameh['cluster'] = frameh.index.str[0]
    frameh = frameh.reset_index()
    return frameh

__Find Cluster Centroids and Cluster Labels__

In [7]:
def centroid_label(frameh, m_pca, m, feature_names, search):
    # most common words in clusters (based on tf-idf not just frequency)
    centroid = dict()
    labels = []
    for c in list(frameh.cluster.unique()):
        ## centroid ## 
        cluster1 = list(frameh[frameh.cluster == c].index.unique())
        # find documents cluster
        m1_pca = m_pca[cluster1,:]
        # take mean vector among all documents
        m1_pca = m1_pca.mean(axis = 0)
        # record mean vector: centroids of each sub cluster
        centroid[c] = m1_pca

        ## labels ##
        # redo mean vector with non-reduced tfidf matrix 
        m1 = m[cluster1,:]
        # take mean vector among all documents
        m1 = m1.mean(axis = 0)
        
        # max values in mean vector 
        lst = []

        for i in np.argsort(np.asarray(m1)[0])[::-1][:6]:
            if feature_names[i] == search: # don't record as label if it is the search
                continue
            lst.append(feature_names[i])
            
        labels.append(lst)
        
    return labels, centroid

__Calculate Silhouette__

In [8]:
def silhouette_individ(frameh, m):
    sil_a = dict()
    for c in list(frameh.cluster.unique()):
        sil_a[c] = dict()
        docs_i = list(frameh[frameh.cluster == c].index.unique())
        for i in docs_i:
            lst = []
            for j in docs_i: 
                if i != j:
                    if type(m) == np.ndarray: # if pca reduced, then ndarray instead of matrix
                        lst.append(np.linalg.norm(m[i]-m[j]))
                    else:
                        lst.append(np.linalg.norm(m[i].toarray()-m[j].toarray()))
            sil_a[c][i] = np.mean(lst)

    sil_b = dict()
    for c in list(frameh.cluster.unique()):
        sil_b[c] = dict()
        docs_in = list(frameh[frameh.cluster == c].index.unique())
        docs_out = list(frameh[frameh.cluster != c].index.unique())
        for i in docs_in:
            lst = []
            for j in docs_out: 
                if type(m) == np.ndarray:
                    lst.append(np.linalg.norm(m[i]-m[j]))
                else:
                    lst.append(np.linalg.norm(m[i].toarray()-m[j].toarray()))
            sil_b[c][i] = np.mean(lst)
            
    return sil_a, sil_b

In [9]:
def silhouette_take_avg(sil):
    avg = []
    for v in sil.values():
        avg.append(list(v.values()))
    avg = [item for sublist in avg for item in sublist]
    avg = [0 if math.isnan(i) else i for i in avg]
    avg = np.mean(avg)
    
    return avg

In [10]:
def silhouette_avg(frameh, m):
    sil_a, sil_b = silhouette_individ(frameh, m)
    avga = silhouette_take_avg(sil_a)
    avgb = silhouette_take_avg(sil_b)

    return (avgb - avga) / max(avgb, avga)

__Clustering__

In [11]:
def find_clusters(k, linkage_matrix, m_pca, m, df, feature_names, search):
    f = fcluster(linkage_matrix, k, criterion = 'maxclust')
    frameh = frame_merge(df, f)
    labels, centroid = centroid_label(frameh, m_pca, m, feature_names, search)

    return frameh, labels, centroid

__Calculate Distortion__

In [12]:
def distortion_calculate(m, centroid, frameh):
    sumd = 0
    for i in list(frameh.index.unique()):
        c = int(frameh[frameh.index == i].cluster)
        sumd += np.linalg.norm(m[i]-centroid[c])
        
    return sumd

__Calculate Distortion, Silhouette at various k values__

In [13]:
def distortion_silhouette(linkage_matrix, m_pca, m, df, feature_names, search):
    # distortion - sum of squared errors between points and its centroid 
    # barely varies with different cluster numbers
    distortion = dict()
    silhouette = dict()

    for k in range(2, min(math.floor(len(df) / 3), 10)): 
        # max # clusters: 1/3 of documents as long as get on average 10 docs per. Else limit to 1/2 of documents. 
        # min # clusters: 2 
        frameh, labels, centroid = find_clusters(k, linkage_matrix, m_pca, m, df, feature_names, search)

        # calculate silhouette 
        silhouette[k] = silhouette_avg(frameh, m_pca)

        # calculate distortion
        sumd = distortion_calculate(m_pca, centroid, frameh)
        # take average 
        distortion[k] = sumd
        
    return distortion, silhouette

In [14]:
def distortion_roc(distortion):
    # relative rate of change 
    roc = []
    for k,v in distortion.items(): 
        if k+1 in distortion:
            roc.append(abs(distortion[k+1] - distortion[k]) / distortion[k])
            
    return roc

__Find K Based on Distortion ROC Elbow__

In [15]:
def find_k(roc):
    # find k using knee method 
    from kneed import KneeLocator
    kn = KneeLocator(range(len(roc)), roc, curve='convex', direction='decreasing')
    k = kn.knee + 1 # index started at 0 
    
    return k 

In [16]:
# plot distortion
#fig, ax = plt.subplots()

#distortion = sorted(distortion.items()) # sorted by key, return a list of tuples
#x, y = zip(*distortion) # unpack a list of pairs into two tuples
#ax.plot(x,y)
#ax.axvline(k, color = 'black')

### Hierarchy: Sub-Clusters

__Linkage Matrix: Understand Node Linkages__

In [17]:
def linkage_df(linkage_matrix):
    # in linkage matrix, indicate the aggregated node for each node pair
    links = pd.DataFrame(linkage_matrix) # using euclidean 
    links.columns = ['source1', 'source2', 'd', 'n']

    links['target'] = 0
    n = 24 
    for i, row in links.iterrows():
        n += 1
        links.at[i,'target'] = n
        
    return links

In [18]:
# flatten irregular nested lists
def flatten(l):
    for el in l:
        if isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

__Find Documents at Various Sub-Clusters__

In [19]:
def merge_docs(merge, source):
    merge = pd.merge(merge, merge[['target', 'docs']], left_on = source, right_on = 'target',  how = 'left')
    merge.docs_x = np.where(merge.docs_x.isnull(), '', merge.docs_x)
    merge.docs_y = np.where(merge.docs_y.isnull(), '', merge.docs_y)
    merge['docs'] = merge[['docs_x', 'docs_y']].values.tolist()
    merge = merge.drop(columns = ['docs_x', 'docs_y'])
    #merge.docs = merge.docs.apply(np.ravel)
    merge = merge.rename(columns = {'target_x':'target'})
    merge.docs = list(merge.docs.apply(lambda row: flatten(row)))
    merge.docs = merge.docs.apply(lambda row: [i for i in row if i != ''])

    return merge

In [58]:
def assign_docs(frameh, links):
    # initial merge between frame ids and source1/source2
    merge = pd.merge(links, frameh[['ids']], left_on = 'source1', right_index = True, how = 'left')
    merge = pd.merge(merge, frameh[['ids']], left_on = 'source2', right_index = True, how = 'left')
    # create single docs list column 
    merge = merge.rename(columns = {'ids_x':'docs1', 'ids_y':'docs2'})
    merge.docs2 = np.where(merge.docs2.isnull(), '', merge.docs2)
    merge.docs1 = np.where(merge.docs1.isnull(), '', merge.docs1)
    merge['docs']= merge[['docs1', 'docs2']].values.tolist()
    merge = merge.drop(columns = ['docs1', 'docs2'])
    # flattern docs list column
    merge.docs = merge.docs.apply(lambda row: [i for i in row if i != ''])
    merge['len'] = merge.docs.apply(lambda row: len(set(row)))

    # loop until have one id per document at node (n)
    while int(merge[merge.target == merge.target.max()].len) != int(merge[merge.target == merge.target.max()].n): 
        print(merge[merge.target == merge.target.max()].len)
        merge = merge_docs(merge, 'source1')
        merge = merge_docs(merge, 'source2')
        merge['len'] = merge.docs.apply(lambda row: len(set(row)))
        
        merge = merge.drop(columns = ['target_y'])

    merge.docs = merge.docs.apply(lambda row: set(row))
    
    return merge 

__Assign Graph Attributes: Docs__

In [21]:
def attributes(G, merge):
    # add docs to each target node so know which docs exist at each target 
    for i in list(merge.target.unique()):
        G.nodes[i]['docs'] = merge[merge.target == i].docs.values[0]

    # add docs to origianl nodes as well 
    ogdocs = pd.merge(links, frameh[['ids']], left_on = 'source1', right_index = True, how = 'left')
    ogdocs = pd.merge(ogdocs, frameh[['ids']], left_on = 'source2', right_index = True, how = 'left')
    for i in range(25):
        if len(ogdocs[ogdocs.source1 == i].ids_x) != 0:
            G.nodes[i]['docs'] = ogdocs[ogdocs.source1 == i].ids_x.values[0]
        if len(ogdocs[ogdocs.source2 == i].ids_y) != 0:
            G.nodes[i]['docs'] = ogdocs[ogdocs.source2 == i].ids_y.values[0]
            
    return G

__Mark Top Level Clusters__

In [22]:
def top_cluster(G, frameh):
    # find top level clusters and mark as such - determined by fcluster above 
    nx.set_node_attributes(G, 0, 'cluster')
    for c in list(frameh.cluster.unique()):
        try:
            c_ids = set(frameh[frameh.cluster == c].ids.unique())
            node = [x for x,y in G.nodes(data=True) if y['docs']==c_ids][0]
        except:
            c_ids = frameh[frameh.cluster == c].ids.unique()
            node = [x for x,y in G.nodes(data=True) if y['docs']==c_ids][0]
        G.nodes[node]['cluster'] = c
        
    return G 

__Create Hierarchy Graph__

In [23]:
def create_graph(links, merge, frameh):
    # add nodes
    G = nx.DiGraph()
    G.add_nodes_from(links.source1)
    G.add_nodes_from(links.source2)
    G.add_nodes_from(links.target)

    # add edges
    subset = links[['source1', 'target']]
    G.add_edges_from([tuple(x) for x in subset.values])
    subset = links[['source2', 'target']]
    G.add_edges_from([tuple(x) for x in subset.values])
    
    # add attributes
    G = attributes(G, merge)
    
    # mark top clusters: attributes
    G = top_cluster(G, frameh)
    
    return G 

In [24]:
def create_hierarchy(linkage_matrix, frameh):
    
    links = linkage_df(linkage_matrix)
    merge = assign_docs(frameh, links)
    G = create_graph(links, merge, frameh)
                     
    return G

# Main Function

In [83]:
def main(reduce):
    
    # read in data
    df = pd.read_pickle('reuters_processed')
    
    distortion_dict = dict()
    silhouette_dict = dict()
    k_dict = dict()
    labels_dict = dict()
    dist_dict = dict()
    graph_dict = dict()
    
    df_final = pd.DataFrame()
    
    for search in ['tin']: #'sugar', 'interest', 'gold'
        df_subset = df[df.categories.map(set([search]).issubset)] 
        df_subset = df_subset.reset_index()
        
        print(search)
        
        # TF-IDF matrix
        tfidf, feature_names = tf_idf(df_subset)
        
        # PCA dimensionality reduction
        if reduce:
            tfidf_unreduced = tfidf.copy()
            tfidf = pca_reduce(tfidf)
                    
        # distances 
        dist, dist_flat = dist_calculate(tfidf)
                
        # linkage matrix
        linkage_matrix = linkage_calculate(dist_flat)
        
        # find K 
        if reduce: 
            # use non-reduced tfidf to find labels. reduced for everything else. 
            distortion_lst, silhouette_lst = distortion_silhouette(linkage_matrix, tfidf, tfidf_unreduced, df_subset,
                                                                   feature_names, search)
            roc = distortion_roc(distortion_lst)
            k = find_k(roc)
            # final flat clusters
            frameh, labels, centroid = find_clusters(k, linkage_matrix, tfidf, tfidf_unreduced, df_subset, feature_names, search)
            distortion = distortion_calculate(tfidf, centroid, frameh)
            silhouette = silhouette_avg(frameh, tfidf)
            
        else:
            # pass tfidf in for both reduced and unreduced arguments
            distortion_lst, silhouette_lst = distortion_silhouette(linkage_matrix, tfidf, tfidf, df_subset, 
                                                                   feature_names, search)
            roc = distortion_roc(distortion_lst)
            k = find_k(roc)

            # final flat clusters
            frameh, labels, centroid = find_clusters(k, linkage_matrix, tfidf, tfidf, df_subset, feature_names, search)
            distortion = distortion_calculate(tfidf, centroid, frameh)
            silhouette = silhouette_avg(frameh, tfidf)
                    
        distortion_dict[search] = distortion
        silhouette_dict[search] = silhouette
        k_dict[search] = k
        labels_dict[search] = labels
        dist_dict[search] = dist
        
        frameh['search'] = search
        df_final = df_final.append(frameh)

    return distortion_dict, silhouette_dict, k_dict, labels_dict, df_final, dist_dict

In [82]:
# create hierarchy graph 
# don't do this in the big loop. far too much processing power
# only do for select searches 
     # but then how will I evaluate?? Maybe don't really and just present as an added bonus...
#G = create_hierarchy(linkage_matrix, frameh)
## need to optimize this. very slow. memory errors. Even for small terms (ex zinc, tin)
## TO DO: modify centroid_label function to get labels for all levels of hierarchy. example in experimention.ipynb

In [ ]:
# TSNE
embed = TSNE(n_components=2).fit_transform(dist_dict['sugar'], 'precomputed')
xs, ys = embed[:, 0], embed[:, 1]

# DataFrame to Plot 
clusters = df_final[df_final.search == 'sugar'].cluster.tolist()
df_vis = pd.DataFrame(dict(x = xs, y = ys, cluster = clusters))
df_vis.cluster = df_vis.cluster - 1 # want clusters to start at 0 

fig, ax = plt.subplots(figsize=(17, 9)) 

groups = df_vis.groupby('cluster')

for name, group in groups:
    ax.scatter(group.x, group.y, label = labels_dict['sugar'][name])
    ax.legend()

In [ ]:
### TO DO: hierarchies 